# Writing `MAF` metrics

## Introduction

This notebook gives instructions on how to write `MAF` metrics, the most common code development task for science groups.

## Notebook preparation

The following is a development style aid; only uncomment if developing the notebook:

In [1]:
# %load_ext lab_black
# %load_ext pycodestyle_magic
# %flake8_on --ignore E501,W505

### Import the required python modules

In [2]:
import rubin_sim
from rubin_sim import maf

Loading BokehJS ...

### Get example data to work with

To test a metric, we will need to use a sample database. Let's use the sample baseline database installed as part of the `rubin_sim` installation process:

In [3]:
opsim_db_fname = rubin_sim.data.get_baseline()
opsim_db_fname

'/data/des91.b/data/neilsen/LSST/devel/rubin_sim_data/sim_baseline/baseline_nexp2_v1.7_10yrs.db'

We will also need an output directory. If you have one you want to use, set it here:

In [4]:
# output_dir = '.'

Otherwise, create a temporary directory that will automatically be cleaned up when this notebook shuts down:

In [5]:
if "output_dir" not in locals():
    from tempfile import TemporaryDirectory

    output_dir_itself = TemporaryDirectory()
    output_dir = output_dir_itself.name

output_dir

'/tmp/tmpamb53q3u'

Finally, lets instantiate an interface to this datadabase for `MAF`:

In [6]:
opsim_db = maf.OpsimDatabase(opsim_db_fname)

# Understanding how metrics are called

To understand how to write metrics, it helps to first know a little bit about slicers.

`MAF` slicers categorize visits in the `opsim` database into groups called "slices." Slices have a "many to many" relationship with visits: a given slice may contain zero or more visits, and a given visit may fall into zero, one, or many slices. For example, the `HealpixSlicer` slices the sky according to [healpixels](https://healpix.jpl.nasa.gov/). Because the sky in any given healpixel can be covered any number of times (including zero), the slice corresponding to a healpixel may contain zero, one, or many visits. A given visit may cover sky in multiple healpixels, and so may be present in multiple slices.

Slicers themselves are python iterators that return tuples with indexes into the array of visits, and slice points (parameters for the slice):

MAF metric objects must all include (among other things): 
- a `colNameArray` member, a `numpy.array` of strings with the names of database columns and any stackers that the metric requires. The `__init__` method of the `rubin_sim.maf.metrics.BaseMetric` includes a `col` named argument which is normally used to set the `colNameArray`. When there is only one element in `colNameArray`, the `colname` member is set with that column name.
- a `run` method that takes two arguments:
  - `dataSlice`, a `numpy.recarray` with one record per visit in a slice, and elements corresponding to (at least) all columns listed in `colNameArray`; and
  - `slicePoint`, a python object (usually a dictionary) defining the parameters of a slice.
  and returns an object that represents the value of the metric for that slice point.

When MAF metrics are computed by calling the `runAll` method of an instance of `rubin_sim.maf.metricBundles.MetricBundleGroup`, the `MetricBundleGroup`:
1. queries the database for any columns required (as specified by the `colNameArray` member of the metric object),
2. computes values for any stackers listen in the metric's `colNameArray` member, and
3. iterates over each slice, calling the metric's `run` method for every slice, and filling the `metricValue` member of the `MetricBundle` with the values returned by `run`.

(This is an over-simplification: the `MetricBundleGroup`'s `runAll` method has various optimizations and other complications that are not important here.)

# Avoid duplicating work

MAF already contains a variety of metrics: before writing a new metric, it is worthwhile to check whether an existing one will do the job. To get a list of metrics that already exist, use:

In [7]:
maf.BaseMetric.help()

AbsMaxMetric
AbsMaxPercentMetric
AbsMeanMetric
AbsMedianMetric
AccumulateCountMetric
AccumulateM5Metric
AccumulateMetric
AccumulateUniformityMetric
ActivityOverPeriodMetric
ActivityOverTimeMetric
AreaSummaryMetric
AveSlewFracMetric
BDParallaxMetric
BaseChildMetric
BaseMoMetric
BinaryMetric
BruteOSFMetric
CampaignLengthMetric
ChipVendorMetric
Coaddm5Metric
Color_AsteroidMetric
CompletenessMetric
CountExplimMetric
CountMetric
CountRatioMetric
CountSubsetMetric
CountUniqueMetric
CrowdingM5Metric
CrowdingMagUncertMetric
DcrPrecisionMetric
DiscoveryMetric
Discovery_DistanceMetric
Discovery_EcLonLatMetric
Discovery_N_ChancesMetric
Discovery_N_ObsMetric
Discovery_RADecMetric
Discovery_TimeMetric
Discovery_VelocityMetric
ExgalM5
ExgalM5_with_cuts
FftMetric
FilterColorsMetric
FracAboveMetric
FracBelowMetric
FullRangeAngleMetric
FullRangeMetric
HighVelocityMetric
HighVelocityNightsMetric
HistogramM5Metric
HistogramMetric
HourglassMetric
IdentityMetric
InstantaneousColorMetric
InterNightGapsMetri

Additional details for each metric can be listed by passing the `doc=True` argument to `help`:

```maf.BaseMetric.help(doc=True)```

If an existing metric does something close to what you want, you can find the source code for guidance thus:

In [8]:
%pinfo maf.SumMetric

Init signature:
maf.SumMetric(
    col=None,
    metricName=None,
    maps=None,
    units=None,
    metricDtype=None,
    badval=-666,
    maskVal=None,
)
Docstring:     
Calculate the sum of a simData column slice.
    
File:           /data/des91.b/data/neilsen/LSST/devel/rubin_sim/rubin_sim/maf/metrics/simpleMetrics.py
Type:           MetricRegistry
Subclasses:     


or dump the whole source code into a jupyter notebook:

In [9]:
%psource maf.SumMetric

class SumMetric(BaseMetric):
    """Calculate the sum of a simData column slice.
    """
    def run(self, dataSlice, slicePoint=None):
        return np.sum(dataSlice[self.colname])


# An exploratory metric

To get a feel for how metrics works, lets create a "dummy" metric that prints information on what arguments it was passed.

In [10]:
import numpy as np


class Test1Metric(maf.BaseMetric):
    def __init__(self):
        # Set the columns we want from the database
        super().__init__(col=["fieldRA", "fieldDec", "HA"], metricName="test1")

    def run(self, dataSlice, slicePoint=None):
        print("--- run ----")
        print(
            f"dataSlice is a {type(dataSlice)} of dtype {dataSlice.dtype} with a shape of {dataSlice.shape}"
        )
        print(f"slicePoint is a {type(slicePoint)} with value: {slicePoint}")

        # Calculate the metric value here, using any values
        # you like from the dataSlice and slicePoint arguments.
        metric_value_for_this_slice = np.random.rand()
        return metric_value_for_this_slice

To apply our metric, we need to create a slicer with which to slice the data. Let's use the `maf.OneDSlicer`, which slices the data into bins according to some value. In this case, we divide the visits into large bins by airmass:

In [11]:
slicer = maf.OneDSlicer(sliceColName="airmass", binMin=1.0, binMax=2.5, binsize=0.5)

Now, instantiate our metric, combine it with our slicer into a `maf.MetricBundle`, and compute the metrics in that bundle with a `maf.MetricBundleGroup`:

In [12]:
metric = Test1Metric()
bundle = maf.MetricBundle(metric, slicer, "", "sample_run")
bgroup = maf.MetricBundleGroup([bundle], dbObj=opsim_db, outDir=output_dir)
bgroup.runAll()

Querying database SummaryAllProps with no constraint for columns ['fieldRA', 'fieldDec', 'airmass', 'observationStartLST'].
Found 2045493 visits
Running:  ['opsim_test1_ONED']
--- run ----
dataSlice is a <class 'numpy.ndarray'> of dtype [('fieldRA', '<f8'), ('fieldDec', '<f8'), ('airmass', '<f8'), ('observationStartLST', '<f8'), ('HA', '<f8')] with a shape of (1954302,)
slicePoint is a <class 'dict'> with value: {'sid': 0, 'binLeft': 1.0}
--- run ----
dataSlice is a <class 'numpy.ndarray'> of dtype [('fieldRA', '<f8'), ('fieldDec', '<f8'), ('airmass', '<f8'), ('observationStartLST', '<f8'), ('HA', '<f8')] with a shape of (79820,)
slicePoint is a <class 'dict'> with value: {'sid': 1, 'binLeft': 1.5}
--- run ----
dataSlice is a <class 'numpy.ndarray'> of dtype [('fieldRA', '<f8'), ('fieldDec', '<f8'), ('airmass', '<f8'), ('observationStartLST', '<f8'), ('HA', '<f8')] with a shape of (11371,)
slicePoint is a <class 'dict'> with value: {'sid': 2, 'binLeft': 2.0}
Completed metric generation

Note that the `dataSlice` `recarray` contained not only the fields we asked for, but two others as well: `airmass` and `observationStartLST`. These are present because the values were needed by the slicer, and by the stacker used to compute the `HA`.

We can look at the values calculated (or in this dummy example, randomly generated) with the `bundle` object:

In [13]:
bundle.metricValues

masked_array(data=[0.07611028882466264, 0.6851090668943071,
                   0.8308709473379822],
             mask=[False, False, False],
       fill_value=0.0)

In this example, the `run` method had access to the `slicePoint` dictionary which included the `binLeft` field showing which bin the slice was. The metric could, therefore, have used it in its calculations. However, doing such would limit the slicers on which the metric could be run to only those slicers with a `binLeft` element in the `slicePoint`.

# Applying a metric to the whole set of visits

Even if we do not want to slice the data, but instead want our `run` method to be passed the entire set of visits, we still need to supply a slicer to satisfy the MAF machinery. The `rubin_sim.maf.slicers.UniSlicer` slicer fills this role, creating one slice with all visits.

In [14]:
slicer = maf.UniSlicer()
bundle = maf.MetricBundle(metric, slicer, "", "unslicer_sample_run")
bgroup = maf.MetricBundleGroup([bundle], dbObj=opsim_db, outDir=output_dir)
bgroup.runAll()

Querying database SummaryAllProps with no constraint for columns ['fieldRA', 'fieldDec', 'observationStartLST'].
Found 2045493 visits
Running:  ['opsim_test1_UNIS']
--- run ----
dataSlice is a <class 'numpy.ndarray'> of dtype [('fieldRA', '<f8'), ('fieldDec', '<f8'), ('observationStartLST', '<f8'), ('HA', '<f8')] with a shape of (2045493,)
slicePoint is a <class 'dict'> with value: {'sid': 0}
Completed metric generation.
Running reduce methods.
Running summary statistics.
Completed.


This time, `airmass` was not required to run the slicer, and it was not one of the fields listed in the `colNameArray` member of our metric, so it was not included in the `dataSlice` `recarray` passed to our metric.

# General metrics

Some metrics are general in their applicability. For example, `maf.MedianMetric` can be applied to any numeric column, and does not use the contents of `slicePoint`s. In this case and those like it, the "work" of the `metric` object can mostly be handled by methods inhereted from `rubin_sim.maf.metrics.BaseMetric`.

Consider the definition of `maf.MedianMetric`:

In [15]:
%psource maf.MedianMetric

class MedianMetric(BaseMetric):
    """Calculate the median of a simData column slice.
    """
    def run(self, dataSlice, slicePoint=None):
        return np.median(dataSlice[self.colname])


How MAF know what column to get the median of? How does `self.colname` get set? 

The `maf.BaseMetric`'s `__init__` method has a `col` argument, and if it is set to a single string rather than a list of strings, the `colname` member takes the value of the one column name. So, the `maf.MedianMetric` must be initialized with the column name, and the `__init__` method of its parent class takes care of assigning the necessary members:

In [16]:
slicer = maf.OneDSlicer(sliceColName="airmass", binMin=1.0, binMax=2.5, binsize=0.5)
metric = maf.MedianMetric(col="moonAlt")
bundle = maf.MetricBundle(metric, slicer, "")
bgroup = maf.MetricBundleGroup([bundle], dbObj=opsim_db, outDir=output_dir)
bgroup.runAll()

Querying database SummaryAllProps with no constraint for columns ['moonAlt', 'airmass'].
Found 2045493 visits
Running:  ['opsim_Median_moonAlt_ONED']
Completed metric generation.
Running reduce methods.
Running summary statistics.
Completed.


Now we can look at the median moon altitudes for each of our large airmass bins:

In [17]:
bundle.metricValues

masked_array(data=[1.3491521979828038, 0.9520903512513406,
                   -1.561837629055115],
             mask=[False, False, False],
       fill_value=0.0)

We can also get the slice points used:

In [18]:
bundle.slicer.slicePoints

{'sid': array([0, 1, 2]), 'bins': array([1. , 1.5, 2. , 2.5])}

# Metric values

A metric's `run` method is called once on each slice generated by the slicer. Although it is usually a numeric value like an `int` or a `float`, it can potentially be any python object.

For example:

In [19]:
class MyBespokeMoon:
    def __init__(self, alt, az):
        self.alt = alt
        self.az = az

    def __repr__(self):
        return f"Moon alt: {self.alt}, az: {self.az}"


class MyWierdMetric(maf.BaseMetric):
    def __init__(self):
        super().__init__(
            col=["moonAlt", "moonAz"], metricName="wierdness", metricDtype="object"
        )

    def run(self, dataSlice, slicePoint=None):
        if len(dataSlice) is None:
            this_slice_value = None
        else:
            this_slice_value = MyBespokeMoon(dataSlice[0].moonAlt, dataSlice[0].moonAz)
        return this_slice_value

When we run this metric, we will get an array of our bespoke object back:

In [20]:
slicer = maf.OneDSlicer(sliceColName="airmass", binMin=1.0, binMax=2.5, binsize=0.5)
metric = MyWierdMetric()
bundle = maf.MetricBundle(metric, slicer, "")
bgroup = maf.MetricBundleGroup([bundle], dbObj=opsim_db, outDir=output_dir)
bgroup.runAll()
bundle.metricValues

Querying database SummaryAllProps with no constraint for columns ['moonAz', 'moonAlt', 'airmass'].
Found 2045493 visits
Running:  ['opsim_wierdness_ONED']
Completed metric generation.
Running reduce methods.
Running summary statistics.
Completed.


masked_array(data=[Moon alt: -32.582749007768506, az: 174.971255360172,
                   Moon alt: 28.616728978690645, az: 321.0903042408855,
                   Moon alt: 53.39480616494672, az: 52.75688276383927],
             mask=[False, False, False],
       fill_value=0,
            dtype=object)

Note that the `metricDtype` argument to the `maf.BaseMetric` initializer declared that the return type is an object.

Returning objects that are not of type `int` or `float` has a serious drawback: the visualization tools in MAF can generally only work with numbers:

In [21]:
bgroup.plotAll()

Plotting figures with "" constraint now.
Plotting complete.


/data/des91.b/data/neilsen/LSST/devel/rubin_sim/rubin_sim/maf/plots/plotHandler.py:518: UserWarning: Cannot plot object metric values with this plotter.
  warnings.warn('Cannot plot object metric values with this plotter.')


# Metrics for use with spatial slicers

If the metric can get all the data it needs from the columns that can be passed in as part of the `dataSlice`, nothing special is required to use them to make a map on the sky.

Some metrics require data on the coordinates of the slice point, and only make sense when appied to a slicer than slices based on position in the sky.
MAF's `rubin_sim.maf.metrics.HealpixSlicer` is an example of such a slicer.
The `HealpixSlicer` creates `slicePoint`s with keys that provide coordinate information on the slice:
- `sid`, the `healpix` id
- `ra`, the RA in radians
- `dec`, the declination in radians
- `nside`, the nside of the healpix pixelization

When combined with values that can be passed in with the `dataSlice`, these values should provide the data needed to, for example, determine which part of the focal plane falls in the slice, or the area covered by the slice.

The `maf.ExgalM5` metric is one example of a metric that requires such coordinate information for from the slicers it can be used with:

In [22]:
%psource maf.ExgalM5

class ExgalM5(BaseMetric):
    """
    Calculate co-added five-sigma limiting depth after dust extinction.

    Uses photUtils to calculate dust extinction.

    Parameters
    ----------
    m5Col : `str`, optional
        Column name for five sigma depth. Default 'fiveSigmaDepth'.
    unit : `str`, optional
        Label for units. Default 'mag'.
    """
    def __init__(self, m5Col='fiveSigmaDepth', metricName='ExgalM5', units='mag',
                 filterCol='filter', **kwargs):
        # Set the name for the dust map to use. This is gathered into the MetricBundle.
        maps = ['DustMap']
        self.m5Col = m5Col
        self.filterCol = filterCol
        super().__init__(col=[self.m5Col, self.filterCol], maps=maps,
                         metricName=metricName, units=units, **kwargs)
        # Set the default wavelength limits for the lsst filters. These are approximately correct.
        dust_properties = Dust_values()
        self.Ax1 = dust_properties.Ax1
        # We wi

This metric demonstrates an additional feature of MAF metrics: the use of maps.
In this case, calculation of the metric requires a dust map, and this is specified in the call to the initializer of the parent class (`super().__init__...`).
When the metric is evaluated, the dictionary supplied in the `slicePoint` argument to `run` includes not only those generated by the slicer, but also `ebv`, the extinction from the dust map.

There are other MAF `map`s available as well, each of which supplements the `slicePoint` dictionary with different values:

In [23]:
maf.BaseMap.help(doc=True)

----  BaseMap  ----
 
 added to SlicePoint:  newkey
----  DustMap  ----

    Compute the E(B-V) for each point in a given spatial distribution of slicePoints.
    
 added to SlicePoint:  ebv
----  GalCoordsMap  ----
None
 added to SlicePoint:  gall,galb
----  StellarDensityMap  ----

    Return the cumulative stellar luminosity function for each slicepoint. Units of stars per sq degree.
    Uses a healpix map of nside=64. Uses the nearest healpix point for other ra,dec values.

    Parameters
    ----------
    startype : `str` ('allstars', 'wdstars')
        Load the luminosity function for all stars ('allstars'), which includes main-sequence stars
        white dwarfs, blue horozontal branch, RR Lyrae, and Cepheids. The 'wdstars' option only includes
        white dwarf stars.
    filtername : `str`
        Filter to use. Options of u,g,r,i,z,y
    
 added to SlicePoint:  starLumFunc_r,starMapBins_r
----  TrilegalDensityMap  ----

    Return the cumulative stellar luminosity function

# What columns and stackers can I use?

You can get a list of columns available in an `opsim` output database using an instance of the `maf.OpsimDatabase` connected to the simulation output of interest.

For older simulations, you need to run:

```
opsim_db = maf.OpsimDatabase('my_old_opsim_output.db`)
opsim_db.get_column_names('SummaryAllProps')
```

For newer simulations, the table queried will need to be `observations` instead of `SummaryAllProps`:

```
opsim_db = maf.OpsimDatabase('my_recent_opsim_output.db`)
opsim_db.get_column_names('observations')
```

This code will print a list of available stackers, and what columns they supply:

In [24]:
import pprint

pprint.pprint({k: v.colsAdded for k, v in maf.BaseStacker.registry.items()})

{'AppMagNullStacker': ['appMag'],
 'AppMagStacker': ['appMag'],
 'BaseDitherStacker': [],
 'BaseMoStacker': [],
 'CoaddStacker': ['coadd'],
 'CometAppMagStacker': ['appMag'],
 'DcrStacker': ['ra_dcr_amp', 'dec_dcr_amp'],
 'EclStacker': ['ecLat', 'ecLon'],
 'EclipticStacker': ['eclipLat', 'eclipLon'],
 'FilterColorStacker': ['rRGB', 'gRGB', 'bRGB'],
 'FiveSigmaStacker': ['m5_simsUtils'],
 'GalacticStacker': ['gall', 'galb'],
 'HexDitherFieldPerNightStacker': ['hexDitherFieldPerNightRa',
                                   'hexDitherFieldPerNightDec'],
 'HexDitherFieldPerVisitStacker': ['hexDitherFieldPerVisitRa',
                                   'hexDitherFieldPerVisitDec'],
 'HexDitherPerNightStacker': ['hexDitherPerNightRa', 'hexDitherPerNightDec'],
 'HourAngleStacker': ['HA'],
 'M5OptimalStacker': ['m5Optimal'],
 'MoMagStacker': ['appMag', 'SNR', 'vis'],
 'NEODistStacker': ['MaxGeoDist', 'NEOHelioX', 'NEOHelioY'],
 'NFollowStacker': ['nObservatories'],
 'NormAirmassStacker': ['norma